Author: Tim Mocking

Contact: t.r.mocking@amsterdamumc.nl

In [ ]:
# File locations
imputed_path = ""
gt_path = ""
figures_path = ""

nilsson_path =  ""
mosmann_path = ""

In [ ]:
# Import all relevant packages
import pandas as pd
import numpy as np
from fcsy import DataFrame
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import seaborn as sns
from utils import load_data, load_Nilsson_data, load_Mosmann_data
from scipy.stats import pearsonr
from statistics import mean, median, stdev
from scipy.stats import wasserstein_distance
from sklearn.metrics import rand_score, adjusted_rand_score, confusion_matrix, \
classification_report, mean_squared_error, recall_score, accuracy_score
from statannot import add_stat_annotation
%load_ext autoreload
%autoreload 2
# Set plotting style
plt.style.use('plotstyle.mplstyle')

In [ ]:
data = load_data(gt_path, imputed_path)
nilsson = load_Nilsson_data(nilsson_path, nilsson_path)
mosmann = load_Mosmann_data(mosmann_path, mosmann_path)

In [ ]:
def calculate_statistics(expression_data, bb_markers, rename, marker_setup, 
                         methods=['CyTOFmerge', 'cyCombine', 'CytoBackBone', 'Infinicyt'],
                         sample_col='sample_id'):
    bin_results = []
    results = []
    for method in methods:
        method_data = expression_data[expression_data['method']==method]
        for sample_id in expression_data[sample_col].unique():
            sample_data = method_data[method_data[sample_col]==sample_id]
            # Filter out unimputed data by cyCombine
            sample_data = sample_data[sample_data['cyCombine_NA']!=True]
            if len(sample_data) == 0:
                print('No sample for '+ method + ' ' + sample_id)
                continue
            sample_data = sample_data.rename(columns=rename)
            imputed_data = sample_data[sample_data['imp_state'] == 1]
            gt_data = sample_data[sample_data['imp_state'] == 0]
            
            assert(len(imputed_data) == len(gt_data))
                        
            # Loop over all imputed channels
            for channel in rename.values():
                # In CytoBackBone we don't have a "dataset" component
                if method != 'CytoBackBone':
                    imputed_ds = imputed_data[imputed_data['dataset']==marker_setup[channel]]
                    gt_ds = gt_data[gt_data['dataset']==marker_setup[channel]]
                else:
                    imputed_ds = imputed_data
                    gt_ds = gt_data
                # Calculate pearson correlation, earth mover's distance
                if len(imputed_ds[channel].unique()) == 1:
                    imputed_ds = imputed_ds.reset_index(drop=True)
                    imputed_ds.loc[0,channel] = 0.00001
                cor = pearsonr(imputed_ds[channel], gt_ds[channel])[0]
                RMSD = mean_squared_error(imputed_ds[channel], gt_ds[channel], squared=False)

                # Calculate statistics per quantile
                if method == 'CyTOFmerge':
                    bin_dict = {'Method':method, 'Channel':channel, 'Sample':sample_id}
                    bins = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
                    for i in range(len(bins)):
                        if i != len(bins)-1:
                            lower = gt_ds[channel].quantile(bins[i])
                            upper = gt_ds[channel].quantile(bins[i+1])
                            gt_bin = gt_ds[(gt_ds[channel] >= lower) & (gt_ds[channel] < upper)]
                            imp_bin = imputed_ds[imputed_ds['original_ID'].isin(list(gt_bin['original_ID']))]
                            bin_cor = pearsonr(gt_bin[channel], imp_bin[channel])[0]
                            bin_dict[str(bins[i])+' - '+str(bins[i+1])] = bin_cor
                    bin_results.append(bin_dict)
                ymin = np.percentile(gt_ds[channel], 0.1)
                if ymin == 0:
                    ymin = 0.01
                ymax = np.percentile(gt_ds[channel], 99.9)
                NRMSD = RMSD / (ymax - ymin)
                EMD = wasserstein_distance(imputed_ds[channel], gt_ds[channel])
                cors = None
                results.append({'Method':method,
                                'Channel':channel, 
                                'r':cor,
                                'EMD':EMD, 
                                'RMSD': RMSD,
                                'NRMSD': NRMSD,
                                'Sample':sample_id,
                                'n_cells':len(gt_data)})
    results = pd.DataFrame(results)
    return results, bin_results

In [ ]:
flow_variable1 = ['FITC-A', 'APC-A', 'BV605-A', 'BV786-A']
flow_variable2 = ['PE-A', 'PE-CF594-A', 'BV711-A', 'PC7-A']
flow_bb = ["HV500c-A", "BUV395-A", "PerCP-Cy5-5-A", "BUV737-A", "BUV496-A", "BV421-A", "APC-R700-A"]

# Define how the different channels should be renamed in plots
flow_rename = {'APC-A':'KLRG1',
               'BV711-A':'TIM-3',
               'FITC-A':'CD57',
               'BV786-A':'CD27',
               'PE-A':'CD28',
               'PE-CF594-A':'CD95',
               'PC7-A':'TIGIT',
               'BV605-A':'PD-1'}

# Define which markers are imputed in which dataset
flow_marker_setup = {'CD57':2,
                     'KLRG1':2,
                     'PD-1':2,
                     'CD27':2,
                     'CD28':1,
                     'CD95':1,
                     'TIM-3':1,
                     'TIGIT':1}

flow_statistics, flow_bins = calculate_statistics(data, flow_bb, flow_rename, flow_marker_setup)

In [ ]:
flow_variable1 = ['Qdot 655-A', 'PE-Cy7-A', 'DAPI-A']
flow_variable2 = ['PE-A', 'QDot 800-A', 'QDot705-A', 'APC-A']
flow_bb = ['PE-Texas Red-A', 'Alexa Fluor 700-A', 'Qdot 605-A', 'PerCP-Cy5-5-A',
           'PE-Cy5-A', 'APC-Cy7-A', 'FITC-A']

# Define how the different channels should be renamed in plots
flow_rename = {'Qdot 655-A': 'CD4',
               'PE-Cy7-A': 'CD10',
               'DAPI-A': 'CD45RA',
               'PE-A': 'CD123',
               'QDot 800-A': 'CD90bio',
               'QDot705-A': 'CD49fpur',
               'APC-A': 'CD110'}
               
# Define which markers are imputed in which dataset
flow_marker_setup = {'CD4':2,
                     'CD10':2,
                     'CD45RA':2,
                     'CD123':1,
                     'CD90bio':1,
                     'CD49fpur':1,
                     'CD110':1}

nilsson_statistics, flow_bins = calculate_statistics(nilsson, flow_bb, flow_rename, flow_marker_setup)

In [ ]:
flow_variable1 = ['Violet H 450/50-A', 'Blue A 710/50-A', 'Blue B 515/20-A',
                     'Red B 710/50-A']
flow_variable2 = ['Green A 780/40-A', 'Violet E 585/42-A', 'Green E 575/25-A',
                     'Red C 660/20-A']
flow_bb = ['Violet G 550/40-A', 'Violet D 605/40-A', 'Green D 610/20-A',
            'Violet C 660/40-A', 'Violet B 705/70-A', 'Violet A 780/60-A',
              'Red A 780/60-A']

# Define how the different channels should be renamed in plots
flow_rename = {'Violet H 450/50-A': 'TNFa',
               'Blue A 710/50-A': 'IL-17A',
               'Blue B 515/20-A': 'CXCR5',
               'Red B 710/50-A': 'IL-2',
               'Green A 780/40-A': 'IFNg',
               'Violet E 585/42-A': 'GZB-SA',
               'Green E 575/25-A': 'CCL4',
               'Red C 660/20-A': 'IL-5'}
               
# Define which markers are imputed in which dataset
flow_marker_setup = {'TNFa':2,
                     'IL-17A':2,
                     'CXCR5':2,
                     'IL-2':2,
                     'IFNg':1,
                     'GZB-SA':1,
                     'CCL4':1,
                     'IL-5':1}

mosmann_statistics, flow_bins = calculate_statistics(mosmann, flow_bb, flow_rename, flow_marker_setup)

# Supplemental Figure 6

In [ ]:
temp = pd.DataFrame(flow_bins)
temp = temp.rename(columns={'Channel':'Marker'})
melt = pd.melt(temp, id_vars=['Method', 'Marker', 'Sample'])
fig = plt.figure(figsize=(3, 4))
sns.lineplot(data=melt[melt['Marker'].isin(['CD95', 'CD57', 'PD-1'])],
                       x='variable', y='value', hue='Marker')

# sns.scatterplot(data=melt, x='variable', y='value')
plt.xlabel('Quantile bin')
plt.ylabel('Pearson correlation')
plt.xticks(rotation=45)

# Figure 2A + 2B

In [ ]:
fig = plt.figure(figsize=(12, 4))

gs = fig.add_gridspec(nrows=1, ncols=5, width_ratios=[0.09, 0.09, 0.09, 0.03, 0.7])

order = ['Infinicyt', 'CyTOFmerge', 'CytoBackBone', 'cyCombine']
palette = ['#f47f2a', '#009cb4', '#ee1d24', '#694893']

ax = fig.add_subplot(gs[0, 0])
plt.title('In-house MM')
sns.boxplot(data=flow_statistics, x='Method', y='EMD', order=order,
            palette=palette, saturation=1, ax=ax)
ax.set_xlabel('')
ax.set_xticklabels([])
ax.legend([], frameon=False)
# ax.spines.left.set_visible(False)
# ax.axes.get_yaxis().set_visible(False)
ax.set_yscale('log')
ax.set_ylim(10e-4, 0.1*10e0)
ax.set_ylabel("Earth mover's distance", fontsize=12)

ax = fig.add_subplot(gs[0, 1])
plt.title('Nilsson_rare')
sns.boxplot(data=nilsson_statistics, x='Method', y='EMD', order=order,
            palette=palette, saturation=1, ax=ax)
ax.set_xlabel('')
ax.set_xticklabels([])
ax.legend([], frameon=False)
ax.spines.left.set_visible(False)
ax.axes.get_yaxis().set_visible(False)
ax.set_yscale('log')
ax.set_ylim(10e-4, 0.1*10e0)
ax.set_ylabel("Earth mover's distance", fontsize=12)

ax = fig.add_subplot(gs[0, 2])
plt.title('Mosmann_rare')
sns.boxplot(data=mosmann_statistics, x='Method', y='EMD', order=order,
            palette=palette, saturation=1, ax=ax)
ax.set_xlabel('')
ax.set_xticklabels([])
ax.spines.left.set_visible(False)
ax.axes.get_yaxis().set_visible(False)
ax.legend([], frameon=False)
ax.set_yscale('log')
ax.set_ylim(10e-4, 0.1*10e0)
ax.set_ylabel("Earth mover's distance", fontsize=12)
add_stat_annotation(ax, data=mosmann_statistics, x='Method', y="EMD", order=order,
                    box_pairs=[("Infinicyt", "CyTOFmerge")],
                    test='t-test_paired', text_format='star', verbose=2, loc='inside')

ax = fig.add_subplot(gs[0, 3])
ax.axes.get_yaxis().set_visible(False)
ax.axes.get_xaxis().set_visible(False)
ax.spines.left.set_visible(False)
ax.spines.bottom.set_visible(False)

ax = fig.add_subplot(gs[0, 4])
sns.barplot(data=flow_statistics, x='Channel', y='EMD', hue='Method', hue_order=order,
            palette=palette, capsize=.05, errwidth=1.25, saturation=1, 
            estimator=median, errorbar=('pi', 50), ax=ax)
ax.set_xlabel('')
ax.legend([], frameon=False)
ax.set_ylabel("Earth mover's distance", fontsize=12)
ax.set_yscale('log')
ax.set_ylim(10e-4, 0.1*10e0)
ax.legend(bbox_to_anchor=(0.5, -0.1), ncol=4, fontsize=12)

plt.subplots_adjust(wspace=0.15, hspace=0)

# Supplemental Figure 3

In [ ]:
fig = plt.figure(figsize=(10, 8))

gs = fig.add_gridspec(nrows=2, ncols=1)

order = ['Infinicyt', 'CyTOFmerge', 'CytoBackBone', 'cyCombine']
palette = ['#f47f2a', '#009cb4', '#ee1d24', '#694893']

ax = fig.add_subplot(gs[0, 0])
sns.barplot(data=nilsson_statistics, x='Channel', y='EMD', hue='Method', hue_order=order,
            palette=palette, capsize=.05, errwidth=1.25, saturation=1, 
            estimator=median, errorbar=('pi', 50), ax=ax)
ax.set_xlabel('')
ax.legend([], frameon=False)
ax.set_ylabel("Earth mover's distance", fontsize=12)
ax.set_yscale('log')
ax.legend(bbox_to_anchor=(0.2, 1), ncol=4, fontsize=12)

ax = fig.add_subplot(gs[1, 0])
sns.barplot(data=mosmann_statistics, x='Channel', y='EMD', hue='Method', hue_order=order,
            palette=palette, capsize=.05, errwidth=1.25, saturation=1, 
            estimator=median, errorbar=('pi', 50), ax=ax)
ax.legend([], frameon=False)
ax.set_yscale('log')
plt.xlabel('')
ax.set_ylabel("Earth mover's distance", fontsize=12)

# Figure 3D + 3E

In [ ]:
fig = plt.figure(figsize=(12, 4))

gs = fig.add_gridspec(nrows=1, ncols=5, width_ratios=[0.09, 0.09, 0.09, 0.03, 0.7])

order = ['Infinicyt', 'CyTOFmerge', 'CytoBackBone', 'cyCombine']
palette = ['#f47f2a', '#009cb4', '#ee1d24', '#694893']

ax = fig.add_subplot(gs[0, 0])
plt.title('In-house MM')
sns.boxplot(data=flow_statistics, x='Method', y='r', order=order,
            palette=palette, saturation=1, ax=ax)
ax.set_xlabel('')
ax.set_xticklabels([])
ax.legend([], frameon=False)
ax.set_ylim(-0.25, 1)
# ax.spines.left.set_visible(False)
# ax.axes.get_yaxis().set_visible(False)
ax.set_ylabel("Pearson correlation", fontsize=12)

ax = fig.add_subplot(gs[0, 1])
plt.title('Nilsson_rare')
sns.boxplot(data=nilsson_statistics, x='Method', y='r', order=order,
            palette=palette, saturation=1, ax=ax)
ax.set_xlabel('')
ax.set_xticklabels([])
ax.legend([], frameon=False)
ax.spines.left.set_visible(False)
ax.axes.get_yaxis().set_visible(False)
ax.set_ylim(-0.25, 1)
ax.set_ylabel("Pearson correlation", fontsize=12)

ax = fig.add_subplot(gs[0, 2])
plt.title('Mosmann_rare')
sns.boxplot(data=mosmann_statistics, x='Method', y='r', order=order,
            palette=palette, saturation=1, ax=ax)
ax.set_xlabel('')
ax.set_xticklabels([])
ax.spines.left.set_visible(False)
ax.axes.get_yaxis().set_visible(False)
ax.legend([], frameon=False)
ax.set_ylim(-0.25, 1)
ax.set_ylabel("Pearson correlation", fontsize=12)

add_stat_annotation(ax, data=mosmann_statistics, x='Method', y="r", order=order,
                    box_pairs=[("Infinicyt", "cyCombine")],
                    test='t-test_paired', text_format='star', verbose=2, loc='inside')

ax = fig.add_subplot(gs[0, 3])
ax.axes.get_yaxis().set_visible(False)
ax.axes.get_xaxis().set_visible(False)
ax.spines.left.set_visible(False)
ax.spines.bottom.set_visible(False)

ax = fig.add_subplot(gs[0, 4])
sns.barplot(data=flow_statistics, x='Channel', y='r', hue='Method', hue_order=order,
            palette=palette, capsize=.05, errwidth=1.25, saturation=1, 
            estimator=median, errorbar=('pi', 50), ax=ax)
ax.set_xlabel('')
ax.legend([], frameon=False)
ax.set_ylabel("Pearson correlation", fontsize=12)
ax.set_ylim(0, 1)
ax.legend(bbox_to_anchor=(0.5, -0.1), ncol=4, fontsize=12)
plt.subplots_adjust(wspace=0.15, hspace=0)

# Supplemental Figure 5

In [ ]:
fig = plt.figure(figsize=(10, 8))

gs = fig.add_gridspec(nrows=2, ncols=1)

order = ['Infinicyt', 'CyTOFmerge', 'CytoBackBone', 'cyCombine']
palette = ['#f47f2a', '#009cb4', '#ee1d24', '#694893']

ax = fig.add_subplot(gs[0, 0])
sns.barplot(data=nilsson_statistics, x='Channel', y='r', hue='Method', hue_order=order,
            palette=palette, capsize=.05, errwidth=1.25, saturation=1, 
            estimator=median, errorbar=('pi', 50), ax=ax)
ax.set_xlabel('')
ax.legend([], frameon=False)
ax.set_ylabel("Pearson correlation", fontsize=12)
ax.legend(bbox_to_anchor=(0.2, 1), ncol=4, fontsize=12)

ax = fig.add_subplot(gs[1, 0])
sns.barplot(data=mosmann_statistics, x='Channel', y='r', hue='Method', hue_order=order,
            palette=palette, capsize=.05, errwidth=1.25, saturation=1, 
            estimator=median, errorbar=('pi', 50), ax=ax)
ax.legend([], frameon=False)
plt.xlabel('')
ax.set_ylabel("Pearson correlation", fontsize=12)

# Supplemental Figure 7

In [ ]:
fig = plt.figure(figsize=(10, 12))

gs = fig.add_gridspec(nrows=3, ncols=2, width_ratios=[0.2, 0.8])

order = ['Infinicyt', 'CyTOFmerge', 'CytoBackBone', 'cyCombine']
palette = ['#f47f2a', '#009cb4', '#ee1d24', '#694893']

ax = fig.add_subplot(gs[0, 0])
sns.boxplot(data=flow_statistics, x='Method', y='NRMSD', order=order,
            palette=palette, saturation=1, ax=ax)
ax.set_xlabel('')
ax.set_xticklabels([])
ax.legend([], frameon=False)
ax.set_ylabel("NRMSD", fontsize=12)

ax = fig.add_subplot(gs[0, 1])
sns.barplot(data=flow_statistics, x='Channel', y='NRMSD', hue='Method', hue_order=order,
            palette=palette, capsize=.05, errwidth=1.25, saturation=1, 
            estimator=median, errorbar=('pi', 50), ax=ax)
ax.set_xlabel('')
ax.legend([], frameon=False)
ax.set_ylabel("NRMSD", fontsize=12)

ax = fig.add_subplot(gs[1, 0])
sns.boxplot(data=nilsson_statistics, x='Method', y='NRMSD', order=order,
            palette=palette, saturation=1, ax=ax)
ax.set_xlabel('')
ax.set_xticklabels([])
ax.legend([], frameon=False)
ax.set_ylabel("NRMSD", fontsize=12)

ax = fig.add_subplot(gs[1, 1])
sns.barplot(data=nilsson_statistics, x='Channel', y='NRMSD', hue='Method', hue_order=order,
            palette=palette, capsize=.05, errwidth=1.25, saturation=1, 
            estimator=median, errorbar=('pi', 50), ax=ax)
ax.set_xlabel('')
ax.legend([], frameon=False)

ax = fig.add_subplot(gs[2, 0])
sns.boxplot(data=mosmann_statistics, x='Method', y='NRMSD', order=order,
            palette=palette, saturation=1, ax=ax)
ax.set_xlabel('')
ax.set_xticklabels([])
ax.legend([], frameon=False)
ax.set_ylabel("NRMSD", fontsize=12)

ax = fig.add_subplot(gs[2, 1])
sns.barplot(data=mosmann_statistics, x='Channel', y='NRMSD', hue='Method', hue_order=order,
            palette=palette, capsize=.05, errwidth=1.25, saturation=1, 
            estimator=median, errorbar=('pi', 50), ax=ax)
ax.set_xlabel('')
ax.legend([], frameon=False)

plt.subplots_adjust(wspace=0.3, hspace=0.2)

ax.legend(bbox_to_anchor=(0.8, -0.1), ncol=4, fontsize=12)